# Play with zarr

## first attempts

In [2]:
import xarray as xr
import obspy
import numpy as np
import pandas as pd

In [ ]:
stream = obspy.read('TA.miniseed')

In [ ]:
trace = stream[0]

In [ ]:
endtime = pd.Timestamp(trace.stats.endtime.datetime)

In [4]:
attrs = dict(trace.stats)

starttime = pd.Timestamp(trace.stats.starttime.datetime).strftime('%Y-%m-%d %X.%f')
endtime = pd.Timestamp(trace.stats.endtime.datetime).strftime('%Y-%m-%d %X.%f')

attrs['starttime'] = starttime
attrs['endtime'] = endtime

del attrs['mseed']

NameError: name 'trace' is not defined

In [6]:
hdata = xr.DataArray(trace.data, attrs=attrs, name='hydrophone data')

In [7]:
hdataset = hdata.to_dataset()

In [8]:
hdataset.to_zarr('test.zarr')

## Open Zarr Dataset

In [9]:
test = xr.open_zarr('../data/test.zarr')

In [10]:
test

<xarray.Dataset>
Dimensions:          (dim_0: 60319998)
Dimensions without coordinates: dim_0
Data variables:
    hydrophone data  (dim_0) int32 dask.array<chunksize=(471250,), meta=np.ndarray>

## Can I open zar dataset from azure

In [14]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

connect_str = "AZURE_STORAGE_CONNECTION_STRING')'DefaultEndpointsProtocol=https;AccountName=lfhydrophone;AccountKey=UohL6H3jDaDLliug0g2Lg5qZZNKWdyxkq2S5KYLE0EFwLjo09auQp48PyhBRKJtYsfjXMmie/d/9+AStLFAniQ==;EndpointSuffix=core.windows.net"


In [17]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
blob_client_instance = blob_service_client.get_blob_client('hydrophonedata', 'test.zar')

In [20]:
import xarray as xr

In [26]:
import pystac

collection = pystac.Collection.from_file(
    "https://lfhydrophone.blob.core.windows.net/hydrophonedata/test.zarr"
)
collection.assets

Exception: Could not read uri https://lfhydrophone.blob.core.windows.net/hydrophonedata/test.zarr

In [28]:
url = "https://lfhydrophone.blob.core.windows.net/hydrophonedata/test.zarr"
xr.open_zarr(url)

/tmp/ipykernel_11455/1816205917.py:2: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  xr.open_zarr(url)


GroupNotFoundError: group not found at path ''

## fsspec and adlfs

In [29]:
import xarray as xr

In [35]:
storage_options={
    'account_name': 'lfhydrophone',
    'account_key': 'UohL6H3jDaDLliug0g2Lg5qZZNKWdyxkq2S5KYLE0EFwLjo09auQp48PyhBRKJtYsfjXMmie/d/9+AStLFAniQ=='
}

ds = xr.open_dataset(
    'abfs://lfhydrophone.blob.core.windows.net/hydrophonedata/test.zarr/',
    backend_kwargs=storage_options,
    engine='zarr'
    )

TypeError: open_dataset() got an unexpected keyword argument 'account_name'